<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ADO-Deliverable" data-toc-modified-id="ADO-Deliverable-1">ADO Deliverable</a></span><ul class="toc-item"><li><span><a href="#Analysis-Questions" data-toc-modified-id="Analysis-Questions-1.1">Analysis Questions</a></span></li></ul></li><li><span><a href="#Importing-Packages-and-Data" data-toc-modified-id="Importing-Packages-and-Data-2">Importing Packages and Data</a></span></li><li><span><a href="#Data-Preprocessing-&amp;-Exploratory-Data-Analysis" data-toc-modified-id="Data-Preprocessing-&amp;-Exploratory-Data-Analysis-3">Data Preprocessing &amp; Exploratory Data Analysis</a></span><ul class="toc-item"><li><span><a href="#Full-DataFrame-Analysis" data-toc-modified-id="Full-DataFrame-Analysis-3.1">Full DataFrame Analysis</a></span></li></ul></li><li><span><a href="#Data-Visualizations" data-toc-modified-id="Data-Visualizations-4">Data Visualizations</a></span><ul class="toc-item"><li><span><a href="#Full-Profile" data-toc-modified-id="Full-Profile-4.1">Full Profile</a></span></li><li><span><a href="#Female-Profile" data-toc-modified-id="Female-Profile-4.2">Female Profile</a></span></li><li><span><a href="#Male-Profile" data-toc-modified-id="Male-Profile-4.3">Male Profile</a></span></li><li><span><a href="#Geographical-Analysis" data-toc-modified-id="Geographical-Analysis-4.4">Geographical Analysis</a></span></li><li><span><a href="#Female-Map" data-toc-modified-id="Female-Map-4.5">Female Map</a></span></li><li><span><a href="#Male-Map" data-toc-modified-id="Male-Map-4.6">Male Map</a></span></li></ul></li></ul></div>

## ADO Deliverable

### Analysis Questions

- **How many males are in the dataset (use gender column)?**
    
    * 498 Males

- **How many females are in the data set (use gender column)?**
    
    * 511 Females

- **How many people are in each of the departments listed (use department col)?**
    
    * Support	106	
    * Product Management	92	
    * Sales	92	
    * Research and Development	91	
    * Business Development	85	
    * Legal	84	
    * Services	84	
    * Engineering	81	
    * Marketing	80	
    * Human Resources	77	
    * Training	69	
    * Accounting	59	
- **How many males are in each department (use gender and department col’s)?**

    * Support	54	
    * Product Management	53	
    * Research and Development	49	
    * Business Development	45	
    * Services	42	
    * Human Resources	40	
    * Legal	39	
    * Engineering	37	
    * Sales	37	
    * Marketing	34	
    * Accounting	32	
    * Training	27		

- **How many females are in each department (use gender and department col’s)?**

    * Sales	55	
    * Support	52	
    * Marketing	46	
    * Legal	45	
    * Engineering	44	
    * Research and Development	42	
    * Services	42	
    * Training	42	
    * Business Development	40	
    * Product Management	39	
    * Human Resources	37	
    * Accounting	27	
- **Who was the earliest person to join the company (use join_date)?**
   *  Arch Wiffield
- **Who was the most recent person to join the company (use join_date)?**
    * Sarita Peppin

## Importing Packages and Data

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import pandas_profiling
import geoip2
import folium
import leaflet
import ipinfo
import branca.colormap 
import flask

In [2]:
file_path = '../data/ADO_data.csv'

In [3]:
df = pd.read_csv(file_path)

## Data Preprocessing & Exploratory Data Analysis 

Setting index to a unique id.

In [4]:
df.set_index("id", inplace=True)

Checking data shape.

In [5]:
df.shape

(1000, 9)

Checking that data types are correct.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   email         1000 non-null   object
 3   gender        1000 non-null   object
 4   ip_address    1000 non-null   object
 5   department    1000 non-null   object
 6   job_title     1000 non-null   object
 7   join_date     1000 non-null   object
 8   company_name  1000 non-null   object
dtypes: object(9)
memory usage: 78.1+ KB


Casting the "join_date" column to datetime.

In [7]:
df["join_date"] = pd.to_datetime(df["join_date"])

Ensuring that my casting worked.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   first_name    1000 non-null   object        
 1   last_name     1000 non-null   object        
 2   email         1000 non-null   object        
 3   gender        1000 non-null   object        
 4   ip_address    1000 non-null   object        
 5   department    1000 non-null   object        
 6   job_title     1000 non-null   object        
 7   join_date     1000 non-null   datetime64[ns]
 8   company_name  1000 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 78.1+ KB


Setting my notebook for optimal data viewing.

In [9]:
pd.set_option('display.max_columns', 500) 

In [10]:
pd.set_option('display.max_rows', 800)

Checking for duplicates and NaN/Null values.

In [11]:
def intitial_eda_checks(df):
    '''
    take a dataframe
    check if there are duplicates
    check if there are nulls
    '''
    if len(df[df.duplicated(keep=False)]) > 0:
        print(df[df.duplicated(keep=False)])
        df.drop_duplicates(keep='first', inplace=True)
        print('Warning! df has been mutated!')
    else:
        print('No duplicates found.')

    if df.isnull().sum().sum() > 0:
        mask_total = df.isnull().sum().sort_values(ascending=False) 
        total = mask_total[mask_total > 0]

        mask_percent = df.isnull().mean().sort_values(ascending=False) 
        percent = mask_percent[mask_percent > 0] 

        missing_data = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percent'])
    
        print(f'Total and Percentage of NaN:\n {missing_data }')
    else: 
        print('No NaN found.')

In [12]:
intitial_eda_checks(df)

No duplicates found.
No NaN found.


Checking column names.

In [13]:
df.columns

Index(['first_name', 'last_name', 'email', 'gender', 'ip_address',
       'department', 'job_title', 'join_date', 'company_name'],
      dtype='object')

Checking Gender Count

 **How many males are in the dataset (use gender column)?**
    
    * 498 Males

 **How many females are in the data set (use gender column)?**
    
    * 511 Females

In [14]:
df['gender'].value_counts()

Female    511
Male      489
Name: gender, dtype: int64

Creating a "binary_gender" column to add to dataframe.

In [15]:
df["binary_gender"] = df["gender"].map({'Female':1, 'Male':0})

In [16]:
df.head()

,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender
id,,,,,,,,,,
1,Timmy,De Bruijn,tdebruijn0@sphinn.com,Female,24.197.167.72,Legal,Graphic Designer,2020-06-12,Browsebug,1
2,Thurston,Rapson,trapson1@cbsnews.com,Male,254.48.92.30,Human Resources,Environmental Specialist,2019-07-22,Flashspan,0
3,Issie,Chowne,ichowne2@hostgator.com,Female,79.251.125.142,Business Development,Budget/Accounting Analyst II,2020-04-14,Zoonder,1
4,Doug,Lindborg,dlindborg3@hibu.com,Male,122.244.152.101,Business Development,Account Executive,2019-10-25,Tavu,0
5,Moss,Knotton,mknotton4@ameblo.jp,Male,34.63.11.216,Business Development,Director of Sales,2020-06-28,Quimm,0


 **How many people are in each of the departments listed (use department col)?**
    
    * Support	106	
    * Product Management	92	
    * Sales	92	
    * Research and Development	91	
    * Business Development	85	
    * Legal	84	
    * Services	84	
    * Engineering	81	
    * Marketing	80	
    * Human Resources	77	
    * Training	69	
    * Accounting	59	

In [17]:
df.groupby(by =["department"]).count()

,first_name,last_name,email,gender,ip_address,job_title,join_date,company_name,binary_gender
department,,,,,,,,,
Accounting,59,59,59,59,59,59,59,59,59
Business Development,85,85,85,85,85,85,85,85,85
Engineering,81,81,81,81,81,81,81,81,81
Human Resources,77,77,77,77,77,77,77,77,77
Legal,84,84,84,84,84,84,84,84,84
Marketing,80,80,80,80,80,80,80,80,80
Product Management,92,92,92,92,92,92,92,92,92
Research and Development,91,91,91,91,91,91,91,91,91
Sales,92,92,92,92,92,92,92,92,92


**How many females are in each department (use gender and department col’s)?**

    * Sales	55	
    * Support	52	
    * Marketing	46	
    * Legal	45	
    * Engineering	44	
    * Research and Development	42	
    * Services	42	
    * Training	42	
    * Business Development	40	
    * Product Management	39	
    * Human Resources	37	
    * Accounting	27	

Creating female mask to grab data.

In [18]:
female = df['binary_gender'] == 1

In [19]:
female_df = df[female]

In [20]:
female_df.groupby("department").count()

,first_name,last_name,email,gender,ip_address,job_title,join_date,company_name,binary_gender
department,,,,,,,,,
Accounting,27,27,27,27,27,27,27,27,27
Business Development,40,40,40,40,40,40,40,40,40
Engineering,44,44,44,44,44,44,44,44,44
Human Resources,37,37,37,37,37,37,37,37,37
Legal,45,45,45,45,45,45,45,45,45
Marketing,46,46,46,46,46,46,46,46,46
Product Management,39,39,39,39,39,39,39,39,39
Research and Development,42,42,42,42,42,42,42,42,42
Sales,55,55,55,55,55,55,55,55,55


**How many males are in each department (use gender and department col’s)?**

    * Support	54	
    * Product Management	53	
    * Research and Development	49	
    * Business Development	45	
    * Services	42	
    * Human Resources	40	
    * Legal	39	
    * Engineering	37	
    * Sales	37	
    * Marketing	34	
    * Accounting	32	
    * Training	27		

Creating male mask to grab data.

In [21]:
male = df['binary_gender'] == 0 

In [22]:
male_df = df[male]

Grouping by "department" and grabbing the count for each department.

In [23]:
male_df.groupby("department").count()

,first_name,last_name,email,gender,ip_address,job_title,join_date,company_name,binary_gender
department,,,,,,,,,
Accounting,32,32,32,32,32,32,32,32,32
Business Development,45,45,45,45,45,45,45,45,45
Engineering,37,37,37,37,37,37,37,37,37
Human Resources,40,40,40,40,40,40,40,40,40
Legal,39,39,39,39,39,39,39,39,39
Marketing,34,34,34,34,34,34,34,34,34
Product Management,53,53,53,53,53,53,53,53,53
Research and Development,49,49,49,49,49,49,49,49,49
Sales,37,37,37,37,37,37,37,37,37


 **Who was the earliest person to join the company (use join_date)?**
   *  Arch Wiffield

In [24]:
df.sort_values(by = "join_date", ascending = True).head(1)

,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender
id,,,,,,,,,,
717,Arch,Wiffield,awiffieldjw@cocolog-nifty.com,Male,181.177.41.189,Research and Development,Civil Engineer,2018-11-01,Chatterbridge,0


**Who was the most recent person to join the company (use join_date)?**
        * Sarita Peppin

In [25]:
df.sort_values(by = "join_date", ascending = False).head(1)

,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender
id,,,,,,,,,,
302,Sarita,Peppin,speppin8d@scribd.com,Female,166.221.220.73,Business Development,Environmental Tech,2021-01-05,Centidel,1


In [26]:
df.groupby(by =["department", "gender"]).count()

first_name  last_name  email  ip_address  \
department               gender                                             
Accounting               Female          27         27     27          27   
                         Male            32         32     32          32   
Business Development     Female          40         40     40          40   
                         Male            45         45     45          45   
Engineering              Female          44         44     44          44   
                         Male            37         37     37          37   
Human Resources          Female          37         37     37          37   
                         Male            40         40     40          40   
Legal                    Female          45         45     45          45   
                         Male            39         39     39          39   
Marketing                Female          46         46     46          46   
                         Male            34         34     34          34   
Product Management       Female          39         39     39          39   
                         Male            53         53     53          53   
Research and Development Female          42         42     42          42   
                         Male            49         49     49          49   
Sales                    Female          55         55     55          55   
                         Male            37         37     37          37   
Services                 Female          42         42     42          42   
                         Male            42         42     42          42   
Support                  Female          52         52     52          52   
                         Male            54         54     54          54   
Training                 Female          42         42     42          42   
                         Male            27         27     27          27   

                                 job_title  join_date  company_name  \
department               gender                                       
Accounting               Female         27         27            27   
                         Male           32         32            32   
Business Development     Female         40         40            40   
                         Male           45         45            45   
Engineering              Female         44         44            44   
                         Male           37         37            37   
Human Resources          Female         37         37            37   
                         Male           40         40            40   
Legal                    Female         45         45            45   
                         Male           39         39            39   
Marketing                Female         46         46            46   
                         Male           34         34            34   
Product Management       Female         39         39            39   
                         Male           53         53            53   
Research and Development Female         42         42            42   
                         Male           49         49            49   
Sales                    Female         55         55            55   
                         Male           37         37            37   
Services                 Female         42         42            42   
                         Male           42         42            42   
Support                  Female         52         52            52   
                         Male           54         54            54   
Training                 Female         42         42            42   
                         Male           27         27            27   

                                 binary_gender  
department               gender                 
Accounting               Female             27  
                         Male       

### Full DataFrame Analysis

Looking at department percentages.

In [27]:
df["department"].value_counts(normalize = True)

Support                     0.106
Sales                       0.092
Product Management          0.092
Research and Development    0.091
Business Development        0.085
Legal                       0.084
Services                    0.084
Engineering                 0.081
Marketing                   0.080
Human Resources             0.077
Training                    0.069
Accounting                  0.059
Name: department, dtype: float64

Looking at job title percentages.

In [28]:
df["job_title"].value_counts(normalize = True)

Editor                                  0.019
Civil Engineer                          0.017
Junior Executive                        0.016
Operator                                0.015
General Manager                         0.013
Executive Secretary                     0.013
Food Chemist                            0.013
Technical Writer                        0.013
Director of Sales                       0.013
Dental Hygienist                        0.012
Senior Developer                        0.012
Cost Accountant                         0.012
Electrical Engineer                     0.011
Business Systems Development Analyst    0.011
Nurse                                   0.011
Information Systems Manager             0.011
Staff Scientist                         0.011
Assistant Professor                     0.011
Occupational Therapist                  0.011
Environmental Tech                      0.011
Clinical Specialist                     0.011
Nuclear Power Engineer            

In [29]:
female_df["job_title"].value_counts(normalize = True)

Junior Executive                        0.025440
Civil Engineer                          0.019569
Director of Sales                       0.017613
Clinical Specialist                     0.017613
Assistant Professor                     0.015656
Staff Scientist                         0.015656
Technical Writer                        0.015656
Cost Accountant                         0.015656
Dental Hygienist                        0.015656
VP Sales                                0.013699
Social Worker                           0.013699
VP Accounting                           0.013699
Information Systems Manager             0.013699
Human Resources Manager                 0.013699
Nurse                                   0.011742
Paralegal                               0.011742
General Manager                         0.011742
Recruiter                               0.011742
GIS Technical Architect                 0.011742
Registered Nurse                        0.011742
Operator            

In [30]:
female_df["department"].value_counts(normalize = True)

Sales                       0.107632
Support                     0.101761
Marketing                   0.090020
Legal                       0.088063
Engineering                 0.086106
Research and Development    0.082192
Training                    0.082192
Services                    0.082192
Business Development        0.078278
Product Management          0.076321
Human Resources             0.072407
Accounting                  0.052838
Name: department, dtype: float64

In [31]:
male_df["department"].value_counts(normalize = True)

Support                     0.110429
Product Management          0.108384
Research and Development    0.100204
Business Development        0.092025
Services                    0.085890
Human Resources             0.081800
Legal                       0.079755
Sales                       0.075665
Engineering                 0.075665
Marketing                   0.069530
Accounting                  0.065440
Training                    0.055215
Name: department, dtype: float64

In [32]:
male_df["job_title"].value_counts(normalize = True)
                                  

Editor                                  0.028630
Operator                                0.018405
Food Chemist                            0.016360
Occupational Therapist                  0.016360
Business Systems Development Analyst    0.016360
Civil Engineer                          0.014315
Payment Adjustment Coordinator          0.014315
Recruiting Manager                      0.014315
Executive Secretary                     0.014315
Structural Analysis Engineer            0.014315
Environmental Specialist                0.014315
General Manager                         0.014315
Electrical Engineer                     0.012270
Product Engineer                        0.012270
Nuclear Power Engineer                  0.012270
Structural Engineer                     0.012270
Research Associate                      0.012270
Analog Circuit Design manager           0.012270
Senior Developer                        0.012270
Environmental Tech                      0.012270
Community Outreach S

## Data Visualizations 

###  Full Profile

In [33]:
full_profile = pandas_profiling.ProfileReport(df) 

In [34]:
full_profile

Saving to html to share with company.

In [35]:
# full_profile.to_file('full_.html')

### Female Profile

In [36]:
pandas_profiling.ProfileReport(female_df)

Saving to html to share with company.

In [37]:
# female_profile.to_file('female_.html')

### Male Profile

In [38]:
male_profile = pandas_profiling.ProfileReport(male_df) 

Saving to html to share with company.

In [39]:
#male_profile.to_file('male_.html')

In [40]:
# dummy_cols = ["gender","department","job_title","company_name"]

In [41]:
# dummied_df = df[dummy_cols]

In [42]:
# dum_df =  pd.get_dummies(dummied_df)

In [43]:
# new_df = pd.concat([df,dum_df ], axis=1)

In [44]:
# new_df.sort_values(by="last_name", inplace = True )

In [45]:
# new_df

### Geographical Analysis 

Grabbing IP addresses and assigning them a geographical location.

In [46]:
ip_df = df['ip_address']

In [47]:
ips = [i for i in ip_df]

In [48]:
def ip_adresses(ip_address):
    access_token = '80ab6ea6d3505a'
    handler = ipinfo.getHandler(access_token)
    details = handler.getDetails(ip_address)
    return details.all

Making sure my function works.

In [49]:
ip_adresses('24.197.167.72')

{'ip': '24.197.167.72',
 'hostname': '024-197-167-072.res.spectrum.com',
 'city': 'Dublin',
 'region': 'Georgia',
 'country': 'US',
 'loc': '32.4593,-82.9381',
 'org': 'AS20115 Charter Communications',
 'postal': '31021',
 'timezone': 'America/New_York',
 'country_name': 'United States',
 'latitude': '32.4593',
 'longitude': '-82.9381'}

In [50]:
locations = [ip_adresses(i) for i in ip_df]

In [51]:
play_data = df

In [52]:
loc_details = pd.DataFrame(locations)

Combining location data with my older dataframe.

In [53]:
combine = pd.merge(play_data, loc_details, left_on='ip_address', right_on='ip')

In [54]:
cols= ["email","ip_address","ip","city"]

Making sure my merge went through.

In [55]:
combine[cols]

,email,ip_address,ip,city
0,tdebruijn0@sphinn.com,24.197.167.72,24.197.167.72,Dublin
1,trapson1@cbsnews.com,254.48.92.30,254.48.92.30,NaN
2,ichowne2@hostgator.com,79.251.125.142,79.251.125.142,Bielefeld
3,dlindborg3@hibu.com,122.244.152.101,122.244.152.101,Ningbo
4,mknotton4@ameblo.jp,34.63.11.216,34.63.11.216,Chicago
...,...,...,...,...
995,jtopingrn@tmall.com,248.106.240.143,248.106.240.143,NaN
996,mgilardonero@jigsy.com,15.123.201.175,15.123.201.175,Atlantic City
997,rportisrp@cisco.com,201.245.201.149,201.245.201.149,Bogotá
998,rcleggrq@jugem.jp,231.177.233.241,231.177.233.241,NaN


Filling NaNs with a place holder "N/A"

In [56]:
combine.fillna("N/A",inplace=True)

Sorting dataframe by last name.

In [57]:
combine.sort_values(by='last_name', ascending = True,inplace=True)

In [58]:
combine.head()

,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender,ip,hostname,city,region,country,loc,org,postal,timezone,country_name,latitude,longitude,bogon
958,Carol-jean,Abell,cabellqm@yellowpages.com,Female,145.220.117.7,Engineering,Sales Representative,2020-07-10,Oyonder,1,145.220.117.7,N/A,Amsterdam,North Holland,NL,"52.3740,4.8897",AS1101 SURFnet bv,1012,Europe/Amsterdam,Netherlands,52.3740,4.8897,N/A
674,Mortimer,Abrashkov,mabrashkoviq@rediff.com,Male,186.122.1.44,Research and Development,Biostatistician III,2020-03-10,Photolist,0,186.122.1.44,host44.186-122-1.telmex.net.ar,Buenos Aires,Buenos Aires F.D.,AR,"-34.6131,-58.3772",AS11664 Techtel LMDS Comunicaciones Interactiv...,1871,America/Argentina/Buenos_Aires,Argentina,-34.6131,-58.3772,N/A
742,Phineas,Ackland,packlandkm@ycombinator.com,Male,30.79.110.167,Engineering,Executive Secretary,2019-03-20,Gabtype,0,30.79.110.167,N/A,Columbus,Ohio,US,"39.9690,-83.0114",N/A,43218,America/New_York,United States,39.9690,-83.0114,N/A
282,Brenn,Acome,bacome7u@elegantthemes.com,Female,193.75.223.232,Sales,Quality Control Specialist,2019-06-24,Avavee,1,193.75.223.232,N/A,Evere,Brussels Capital,BE,"50.8744,4.3990",AS5432 Proximus NV,1140,Europe/Brussels,Belgium,50.8744,4.3990,N/A
280,Elwood,Adamou,eadamou7s@posterous.com,Male,229.158.193.75,Support,Environmental Tech,2019-07-02,Twinder,0,229.158.193.75,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,True


In [59]:
# main_profile = pandas_profiling.ProfileReport(combine) 

In [60]:
# main_profile

In [61]:
# main_profile.to_file('full_.html')

In [62]:
intitial_eda_checks(combine)

No duplicates found.
No NaN found.


In [63]:
combine['country'].unique()

array(['NL', 'AR', 'US', 'BE', 'N/A', 'CN', 'DE', 'CZ', 'EC', 'BR', 'RU',
       'ID', 'TW', 'ZA', 'JP', 'MX', 'KR', 'UZ', 'FR', 'GB', 'CO', 'VN',
       'IQ', 'DK', 'ES', 'SE', 'CA', 'AU', 'IR', 'FI', 'MA', 'TR', 'CL',
       'IT', 'PL', 'NO', 'CR', 'VE', 'IN', 'TN', 'UA', 'RO', 'CH', 'GR',
       'SG', 'SK', 'ZM', 'AF', '', 'GE', 'IE', 'TH', 'SI', 'BG', 'TZ',
       'PH', 'KM', 'PE', 'GP', 'PT', 'SA', 'AE', 'AT', 'LS', 'EG', 'DZ',
       'HU', 'NZ', 'UY', 'BO', 'HR', 'BI', 'QA', 'AO', 'MN', 'MY', 'HK',
       'LV', 'KZ', 'MO', 'SD', 'KG', 'CI'], dtype=object)

There is alot of location data but its too granular, by making it a bit more macro it become more digestible.

In [64]:
combine['Continent']  = combine['country'].map({'NL':'EUROPE','AR':'SOUTH AMERICA', 'US':'NORTH AMERICA', 'BE':'EUROPE', 'N/A':'N/A','CN':'ASIA', 'DE':'EUROPE', 'CZ':'EUROPE', 'EC':'SOUTH AMERICA','BR':'SOUTH AMERICA', 'RU':'EUROPE',
      'ID': 'ASIA', 'TW':'ASIA', 'ZA':'AFRICA','JP':'ASIA', 'MX':'CENTRAL AMERICA', 'KR':'ASIA', 'UZ':'EUROPE', 'FR':'EUROPE', 'GB':'EUROPE', 'CO':'SOUTH AMERICA', 'VN':'ASIA',
       'IQ':'ASIA', 'DK':'EUROPE', 'ES':'EUROPE', 'SE':'EUROPE', 'CA':'NORTH AMERICA', 'AU':'AUSTRALIA & OCEANIA', 'IR':'ASIA', 'FI':'EUROPE', 'MA':'AFRICA', 'TR':'EUROPE', 'CL':'SOUTH AMERICA',
       'IT':'EUROPE', 'PL':'EUROPE', 'NO':'EUROPE', 'CR':'CENTRAL AMERICA', 'VE':'SOUTH AMERICA', 'IN':'ASIA', 'TN':'AFRICA', 'UA':'EUROPE', 'RO':'EUROPE', 'CH':'EUROPE', 'GR':'EUROPE',
       'SG':'ASIA', 'SK': 'EUROPE', 'ZM':'AFRICA', 'AF':'ASIA', '':'', 'GE':'EUROPE', 'IE':'EUROPE', 'TH':'ASIA', 'SI':'EUROPE', 'BG':'EUROPE', 'TZ':'AFRICA',
       'PH':'ASIA', 'KM':'AFRICA', 'PE':'SOUTH AMERICA', 'GP':'CENTRAL AMERICA', 'PT':'EUROPE', 'SA':'ASIA', 'AE':'ASIA', 'AT':'EUROPE', 'LS':'AFRICA', 'EG':'AFRICA', 'DZ':'AFRICA',
       'HU':'EUROPE', 'NZ':'AUSTRALIA & OCEANIA', 'UY':'SOUTH AMERICA', 'BO':'SOUTH AMERICA', 'HR':'EUROPE', 'BI':'AFRICA', 'QA':'ASIA', 'AO':'AFRICA', 'MN':'ASIA', 'MY':'ASIA', 'HK':'ASIA',
       'LV':'EUROPE', 'KZ':'EUROPE', 'MO':'ASIA', 'SD':'AFRICA', 'KG':'EUROPE', 'CI':'AFRICA'})

Making sure my mapping worked, now we can see what continents most customers are based in. Once we know this we can target digital marketing accordingly. 

In [65]:
combine['Continent'].value_counts()

NORTH AMERICA          383
EUROPE                 201
ASIA                   174
N/A                    152
SOUTH AMERICA           45
AFRICA                  24
CENTRAL AMERICA         11
AUSTRALIA & OCEANIA      8
                         2
Name: Continent, dtype: int64

In [66]:
combine.groupby(['gender','Continent',]).count()

first_name  last_name  email  ip_address  \
gender Continent                                                       
Female                               1          1      1           1   
       AFRICA                       11         11     11          11   
       ASIA                         96         96     96          96   
       AUSTRALIA & OCEANIA           4          4      4           4   
       CENTRAL AMERICA               5          5      5           5   
       EUROPE                       99         99     99          99   
       N/A                          86         86     86          86   
       NORTH AMERICA               181        181    181         181   
       SOUTH AMERICA                28         28     28          28   
Male                                 1          1      1           1   
       AFRICA                       13         13     13          13   
       ASIA                         78         78     78          78   
       AUSTRALIA & OCEANIA           4          4      4           4   
       CENTRAL AMERICA               6          6      6           6   
       EUROPE                      102        102    102         102   
       N/A                          66         66     66          66   
       NORTH AMERICA               202        202    202         202   
       SOUTH AMERICA                17         17     17          17   

                            department  job_title  join_date  company_name  \
gender Continent                                                             
Female                               1          1          1             1   
       AFRICA                       11         11         11            11   
       ASIA                         96         96         96            96   
       AUSTRALIA & OCEANIA           4          4          4             4   
       CENTRAL AMERICA               5          5          5             5   
       EUROPE                       99         99         99            99   
       N/A                          86         86         86            86   
       NORTH AMERICA               181        181        181           181   
       SOUTH AMERICA                28         28         28            28   
Male                                 1          1          1             1   
       AFRICA                       13         13         13            13   
       ASIA                         78         78         78            78   
       AUSTRALIA & OCEANIA           4          4          4             4   
       CENTRAL AMERICA               6          6          6             6   
       EUROPE                      102        102        102           102   
       N/A                          66         66         66            66   
       NORTH AMERICA               202        202        202           202   
       SOUTH AMERICA                17         17         17            17   

                            binary_gender   ip  hostname  city  region  \
gender Continent                                                         
Female                                  1    1         1     1       1   
       AFRICA                          11   11        11    11      11   
       ASIA                            96   96        96    96      96   
       AUSTRALIA & OCEANIA              4    4         4     4       4   
       CENTRAL AMERICA                  5    5         5     5       5   
       EUROPE                          99   99        99    99      99   
       N/A                             86   86        86    86      86   
       NORTH AMERICA                  181  181       181   181     181   
       SOUTH AMERICA                   28   28        28    28      28   
Male                                    1    1         1     1       1   
       AFRICA                          13   13        13    13      13   
       ASIA                            78   78        78    78  

In [67]:
female_ = combine['binary_gender'] == 1

In [68]:
female_combine = combine[female_]

In [69]:
female_combine.groupby("Continent").count()

,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender,ip,hostname,city,region,country,loc,org,postal,timezone,country_name,latitude,longitude,bogon
Continent,,,,,,,,,,,,,,,,,,,,,,,
,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
AFRICA,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
ASIA,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96
AUSTRALIA & OCEANIA,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
CENTRAL AMERICA,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
EUROPE,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
N/A,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86
NORTH AMERICA,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181,181
SOUTH AMERICA,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28


In [70]:
female_combine.groupby(["country_name","Continent"]).count()

,,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender,ip,hostname,city,region,country,loc,org,postal,timezone,latitude,longitude,bogon
country_name,Continent,,,,,,,,,,,,,,,,,,,,,,
Algeria,AFRICA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Argentina,SOUTH AMERICA,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Australia,AUSTRALIA & OCEANIA,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Belgium,EUROPE,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Brazil,SOUTH AMERICA,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
Bulgaria,EUROPE,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Burundi,AFRICA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Canada,NORTH AMERICA,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
Chile,SOUTH AMERICA,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [71]:
male_ = combine['binary_gender'] == 0

In [72]:
male_combine = combine[male_]

In [73]:
male_combine.groupby("Continent").count()

,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender,ip,hostname,city,region,country,loc,org,postal,timezone,country_name,latitude,longitude,bogon
Continent,,,,,,,,,,,,,,,,,,,,,,,
,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
AFRICA,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
ASIA,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78,78
AUSTRALIA & OCEANIA,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
CENTRAL AMERICA,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
EUROPE,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102,102
N/A,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66
NORTH AMERICA,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202,202
SOUTH AMERICA,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17


In [74]:
male_combine.groupby(["country_name","Continent"]).count()

,,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender,ip,hostname,city,region,country,loc,org,postal,timezone,latitude,longitude,bogon
country_name,Continent,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,ASIA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Angola,AFRICA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Argentina,SOUTH AMERICA,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
Australia,AUSTRALIA & OCEANIA,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Austria,EUROPE,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Belgium,EUROPE,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Bolivia,SOUTH AMERICA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Brazil,SOUTH AMERICA,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
Canada,NORTH AMERICA,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9


Trying to see what is going on with all the "N/A" values, turns out they are from a bogons which means they cannot be traced. This also is dangerous from a cybersecurity standpoint because bogons are typical used by cyber criminals.

In [75]:
continent_empty_mask = combine['Continent'] == ""

In [76]:
continent_null_mask = combine['Continent'] == 'N/A'

In [77]:
bogon_ips = combine[continent_null_mask]

In [78]:
empty_mask = combine[continent_empty_mask]

In [79]:
combine["Continent"].replace("","N/A",inplace=True)

In [80]:
 combine[continent_empty_mask]

,first_name,last_name,email,gender,ip_address,department,job_title,join_date,company_name,binary_gender,ip,hostname,city,region,country,loc,org,postal,timezone,country_name,latitude,longitude,bogon,Continent
751,Seth,Givens,sgivenskv@sciencedaily.com,Male,160.234.117.69,Services,Structural Engineer,2020-02-14,Meemm,0,160.234.117.69,N/A,,N/A,,"0.0000,0.0000",N/A,N/A,N/A,N/A,0.0000,0.0000,N/A,N/A
581,Corette,Jancso,cjancsog5@mysql.com,Female,102.215.161.218,Marketing,Assistant Professor,2019-12-17,Cogilith,1,102.215.161.218,N/A,,N/A,,"0.0000,0.0000",N/A,N/A,N/A,N/A,0.0000,0.0000,N/A,N/A


Setting the central coordinates top map our IP addresses.

In [81]:
Manhattan_coords = [40.7831, -73.9712]

my_map = folium.Map(location = Manhattan_coords, zoom_start = 13)

Cleaning the coords and mapping them.

nan Error fix

In [82]:
from numpy import nan
nan == nan

False

In [83]:
combine["Continent"].replace(nan,"N/A",inplace=True)

In [84]:
combine["Continent"].fillna("N/A")

958           EUROPE
674    SOUTH AMERICA
742    NORTH AMERICA
282           EUROPE
280              N/A
           ...      
633             ASIA
234           EUROPE
459              N/A
109    NORTH AMERICA
430    NORTH AMERICA
Name: Continent, Length: 1000, dtype: object

In [85]:
color_dict = {'NORTH AMERICA': 'crimson',         
'EUROPE':  'darkcyan',           
'ASIA':  'darkorange',                                 
'SOUTH AMERICA':'pink',  
'AFRICA':   'mediumslateblue',            
'CENTRAL AMERICA':  'green',    
'AUSTRALIA & OCEANIA' :'black',
             'N/A':'yellow',
             None:'yellow',
             '': 'yellow'}

In [86]:
for index, row in combine.iterrows():
    continent = row['Continent']
    cluster = row['loc']
    clean_coords = [row['latitude'], row['longitude']]
    if cluster != "N/A":
        folium.CircleMarker(
        clean_coords, #passing coords into folium to be marked
        radius= 5,
        color='b',
        threshold_scale=[0,1,2,3],
        fill_color= color_dict[continent],
        fill=True,
        fill_opacity=0.7
        ).add_to(my_map)


In [87]:
# #specify the min and max values of your data
# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
# colormap = colormap.to_step(index=[0,1,2,3])
# colormap.caption = 'Customers by Continent'
# colormap.add_to(my_map)

Now we can see where our customers are based out of.

In [88]:
my_map 

Saving the map to html for viewing.

In [91]:
# my_map.save('map_.png')

### Female Map

In [92]:
Manhattan_coords = [40.7831, -73.9712]

my_map_female = folium.Map(location = Manhattan_coords, zoom_start = 13)

In [93]:
for index, row in female_combine.iterrows():
    continent = row['Continent']
    cluster = row['loc']
    clean_coords = [row['latitude'], row['longitude']]
    if cluster != "N/A":
        folium.CircleMarker(
        clean_coords, #passing coords into folium to be marked
        radius= 5,
        color='b',
        threshold_scale=[0,1,2,3],
        fill_color= color_dict[continent],
        fill=True,
        fill_opacity=0.7
        ).add_to(my_map_female)


In [94]:
my_map_female

In [95]:
#my_map_female.save('map_female_.png')

### Male Map

In [97]:
Manhattan_coords = [40.7831, -73.9712]

my_map_male = folium.Map(location = Manhattan_coords, zoom_start = 13)

In [98]:
for index, row in male_combine.iterrows():
    continent = row['Continent']
    cluster = row['loc']
    clean_coords = [row['latitude'], row['longitude']]
    if cluster != "N/A":
        folium.CircleMarker(
        clean_coords, #passing coords into folium to be marked
        radius= 5,
        color='b',
        threshold_scale=[0,1,2,3],
        fill_color= color_dict[continent],
        fill=True,
        fill_opacity=0.7
        ).add_to(my_map_male)

In [99]:
my_map_male

In [100]:
# my_map_male.save('map_male_.png')